# Space Apps Challenge 2019

# Planets Near and Far

## Chasers of the Lost Data
##### https://2019.spaceappschallenge.org/challenges/planets-near-and-far/raiders-lost-data/details

## Group: ARCADE GT
### Project: SPACE DS
### Data Set: Meteorite Landings

##### https://2019.spaceappschallenge.org/challenges/planets-near-and-far/raiders-lost-data/teams/arcade-gt/project

In [1]:
import os
from sklearn.impute import SimpleImputer
import impyute as impy
import numpy as np, pandas as pd, tensorflow as tf, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model

Using TensorFlow backend.


In [2]:
df_3 = pd.read_csv("NASA/Meteorite_Landings.csv", encoding = "latin-1") #Load the Data


##### *First Step: See the data*

In [3]:
df_3 #Fireball_And_Bolide_Reports

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,01/01/1880 12:00:00 AM,50.77500,6.08333,"(50.775, 6.08333)"
1,Aarhus,2,Valid,H6,720.0,Fell,01/01/1951 12:00:00 AM,56.18333,10.23333,"(56.18333, 10.23333)"
2,Abee,6,Valid,EH4,107000.0,Fell,01/01/1952 12:00:00 AM,54.21667,-113.00000,"(54.21667, -113.0)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,01/01/1976 12:00:00 AM,16.88333,-99.90000,"(16.88333, -99.9)"
4,Achiras,370,Valid,L6,780.0,Fell,01/01/1902 12:00:00 AM,-33.16667,-64.95000,"(-33.16667, -64.95)"
5,Adhi Kot,379,Valid,EH4,4239.0,Fell,01/01/1919 12:00:00 AM,32.10000,71.80000,"(32.1, 71.8)"
6,Adzhi-Bogdo (stone),390,Valid,LL3-6,910.0,Fell,01/01/1949 12:00:00 AM,44.83333,95.16667,"(44.83333, 95.16667)"
7,Agen,392,Valid,H5,30000.0,Fell,01/01/1814 12:00:00 AM,44.21667,0.61667,"(44.21667, 0.61667)"
8,Aguada,398,Valid,L6,1620.0,Fell,01/01/1930 12:00:00 AM,-31.60000,-65.23333,"(-31.6, -65.23333)"
9,Aguila Blanca,417,Valid,L,1440.0,Fell,01/01/1920 12:00:00 AM,-30.86667,-64.55000,"(-30.86667, -64.55)"


In [4]:
#Search Null
np.sum(df_3.isnull())

name              0
id                0
nametype          0
recclass          0
mass (g)        131
fall              0
year            291
reclat         7315
reclong        7315
GeoLocation    7315
dtype: int64

In [5]:
df_3_nn = df_3.dropna(how = "any")
np.sum(df_3_nn.isnull())

name           0
id             0
nametype       0
recclass       0
mass (g)       0
fall           0
year           0
reclat         0
reclong        0
GeoLocation    0
dtype: int64

##### *Second Step: Clean the data*

In [6]:
print("dimensiones con nulls =", df_3.shape)
print("dimensiones sin nulls =", df_3_nn.shape)

dimensiones con nulls = (45716, 10)
dimensiones sin nulls = (38115, 10)


###### Delete unnecesary columns

In [8]:
df_3_nn.columns

Index(['name', 'id', 'nametype', 'recclass', 'mass (g)', 'fall', 'year',
       'reclat', 'reclong', 'GeoLocation'],
      dtype='object')

In [9]:
del df_3_nn['name']
del df_3_nn['id']
del df_3_nn['nametype']

In [13]:
data_y = df_3_nn["mass (g)"]
data_y.shape

(38115,)

***
### Categorical Data

* recclass
* fall
* year
* GeoLocation

#### One Hot Encoding GeoLocation

In [14]:
print(df_3_nn['GeoLocation'].value_counts())

(0.0, 0.0)                6186
(-71.5, 35.66667)         4760
(-84.0, 168.0)            3040
(-72.0, 26.0)             1505
(-79.68333, 159.75)        657
                          ... 
(-85.16782, -94.72788)       1
(34.6, 116.75)               1
(-72.98917, 160.405)         1
(19.11067, 54.81633)         1
(34.00056, -115.72972)       1
Name: GeoLocation, Length: 16907, dtype: int64


In [15]:
df_3_nn['GeoLocation'] = pd.Categorical(df_3_nn['GeoLocation'])
df_3_nn['GeoLocation'] = df_3_nn['GeoLocation'].cat.codes 
data_encoded_GeoLocation = to_categorical(df_3_nn['GeoLocation'])
data_encoded_GeoLocation.shape

C:\Users\Jose\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Jose\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(38115, 16907)

#### One Hot Encoding year

In [16]:
print(df_3_nn['year'].value_counts())

01/01/1979 12:00:00 AM    3045
01/01/1988 12:00:00 AM    2295
01/01/1998 12:00:00 AM    2147
01/01/2003 12:00:00 AM    1754
01/01/2006 12:00:00 AM    1616
                          ... 
12/28/0860 12:00:00 AM       1
01/01/1751 12:00:00 AM       1
01/01/1797 12:00:00 AM       1
12/23/1490 12:00:00 AM       1
01/01/1809 12:00:00 AM       1
Name: year, Length: 253, dtype: int64


In [17]:
df_3_nn['year'] = pd.Categorical(df_3_nn['year'])
df_3_nn['year'] = df_3_nn['year'].cat.codes 
data_encoded_year = to_categorical(df_3_nn['year'])
data_encoded_year.shape

C:\Users\Jose\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Jose\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(38115, 253)

#### One Hot Encoding recclass

In [18]:
print(df_3_nn['recclass'].value_counts())

L6                       7519
H5                       6243
H6                       3898
H4                       3880
L5                       3264
                         ... 
Acapulcoite/lodranite       1
L5-7                        1
H3.7-6                      1
H(L)3                       1
L(LL)3.5-3.7                1
Name: recclass, Length: 422, dtype: int64


In [19]:
df_3_nn['recclass'] = pd.Categorical(df_3_nn['recclass'])
df_3_nn['recclass'] = df_3_nn['recclass'].cat.codes 
data_encoded_recclass = to_categorical(df_3_nn['recclass'])
data_encoded_recclass.shape

C:\Users\Jose\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Jose\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(38115, 422)

In [20]:
#### One Hot Encoding fall

In [21]:
print(df_3_nn['fall'].value_counts())

Found    37050
Fell      1065
Name: fall, dtype: int64


In [22]:
df_3_nn['fall'] = pd.Categorical(df_3_nn['fall'])
df_3_nn['fall'] = df_3_nn['fall'].cat.codes 
data_encoded_fall = to_categorical(df_3_nn['fall'])
data_encoded_fall.shape

C:\Users\Jose\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Jose\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(38115, 2)

In [23]:
data_encoded_fall

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

##### Prepare data

In [24]:
col_name = df_3_nn.columns
col_name

Index(['recclass', 'mass (g)', 'fall', 'year', 'reclat', 'reclong',
       'GeoLocation'],
      dtype='object')

In [49]:
col_x = col_name.drop(['fall','recclass', "year",'GeoLocation', 'mass (g)'])
col_x

Index(['reclat', 'reclong'], dtype='object')

In [50]:
data_x = df_3_nn[col_x]
data_x.shape

(38115, 2)

In [46]:
data_x = pd.concat([data_x, pd.DataFrame(data_encoded_fall, columns = ["Found", "Fell"])], axis=1)
data_x.shape

(45213, 4)

In [51]:
data_x.head()

,reclat,reclong
0,50.77500,6.08333
1,56.18333,10.23333
2,54.21667,-113.00000
3,16.88333,-99.90000
4,-33.16667,-64.95000


In [60]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size = 0.1, random_state = 0)

### Build the Model with Multilayer Perceptron

In [61]:
model_1 = Sequential()
model_1.add(Dense(128, input_shape = (2, ), activation = "relu"))
model_1.add(Dense(64, activation = "relu"))
model_1.add(Dense(32, activation = "relu"))
model_1.add(Dense(1, activation = "softmax"))
model_1.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 128)               384       
_________________________________________________________________
dense_22 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_23 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 33        
Total params: 10,753
Trainable params: 10,753
Non-trainable params: 0
_________________________________________________________________


In [62]:
monitor_val_acc_1 = EarlyStopping(monitor = "val_acc", patience = 3)
modelCheckpoint_1 = ModelCheckpoint("beer_batch_128.hdf5", save_best_only = True)

history_1 = model_1.fit(x_train, y_train, epochs = 100, batch_size = 128, validation_split = 0.1, callbacks = [monitor_val_acc_1, modelCheckpoint_1])
accuracy_1 = model_1.evaluate(x_test, y_test)[1]
print('Accuracy:', accuracy_1)

Train on 30872 samples, validate on 3431 samples
Epoch 1/100
30872/30872 [==============================] - 3s 83us/step - loss: -247197.4848 - acc: 0.0022 - val_loss: -106066.3523 - val_acc: 0.0023
Epoch 2/100
30872/30872 [==============================] - 1s 23us/step - loss: -247197.4781 - acc: 0.0022 - val_loss: -106066.3523 - val_acc: 0.0023
Epoch 3/100
30872/30872 [==============================] - 1s 23us/step - loss: -247197.4920 - acc: 0.0022 - val_loss: -106066.3523 - val_acc: 0.0023
Epoch 4/100
3812/3812 [==============================] - 0s 31us/step
Accuracy: 0.0018363064008394543


In [63]:
monitor_val_acc_2 = EarlyStopping(monitor = "val_acc", patience = 3)
modelCheckpoint_2 = ModelCheckpoint("beer_no_batch.hdf5", save_best_only = True)

history_2 = model_1.fit(x_train, y_train, epochs = 100, validation_split = 0.2, callbacks = [monitor_val_acc_2, modelCheckpoint_2])
accuracy_2 = model_1.evaluate(x_test, y_test)[1]
print('Accuracy:', accuracy_2)

Train on 27442 samples, validate on 6861 samples
Epoch 1/100
27442/27442 [==============================] - 3s 101us/step - loss: -254795.1153 - acc: 0.0023 - val_loss: -146233.3395 - val_acc: 0.0019
Epoch 2/100
27442/27442 [==============================] - 3s 100us/step - loss: -254795.1127 - acc: 0.0023 - val_loss: -146233.3395 - val_acc: 0.0019
Epoch 3/100
27442/27442 [==============================] - 3s 92us/step - loss: -254795.1160 - acc: 0.0023 - val_loss: -146233.3395 - val_acc: 0.0019
Epoch 4/100
3812/3812 [==============================] - 0s 33us/step
Accuracy: 0.0018363064008394543


In [71]:


norm_model = Sequential()
norm_model.add(Dense(128, input_shape = (2, ), activation = "relu"))
norm_model.add(BatchNormalization())
norm_model.add(Dense(64, activation = "relu"))
norm_model.add(BatchNormalization())
norm_model.add(Dense(32, activation = "relu"))
norm_model.add(BatchNormalization())
norm_model.add(Dense(1, activation = "softmax"))
norm_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
norm_model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 128)               384       
_________________________________________________________________
batch_normalization_10 (Batc (None, 128)               512       
_________________________________________________________________
dense_38 (Dense)             (None, 64)                8256      
_________________________________________________________________
batch_normalization_11 (Batc (None, 64)                256       
_________________________________________________________________
dense_39 (Dense)             (None, 32)                2080      
_________________________________________________________________
batch_normalization_12 (Batc (None, 32)                128       
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 33        
Total para

In [72]:


monitor_val_acc_3 = EarlyStopping(monitor = "val_acc", patience = 3)
modelCheckpoint_3 = ModelCheckpoint("beer_norm_no_batch.hdf5", save_best_only = True)

history_3 = norm_model.fit(x_train, y_train, epochs = 100, validation_split = 0.2, callbacks = [monitor_val_acc_3, modelCheckpoint_3])
accuracy = norm_model.evaluate(x_test, y_test)[1]
print('Accuracy:', accuracy)



Train on 27442 samples, validate on 6861 samples
Epoch 1/100
27442/27442 [==============================] - 8s 296us/step - loss: -254795.1173 - acc: 0.0023 - val_loss: -146233.3395 - val_acc: 0.0019
Epoch 2/100
27442/27442 [==============================] - 4s 153us/step - loss: -254795.1117 - acc: 0.0023 - val_loss: -146233.3395 - val_acc: 0.0019
Epoch 3/100
27442/27442 [==============================] - 4s 156us/step - loss: -254795.1123 - acc: 0.0023 - val_loss: -146233.3395 - val_acc: 0.0019
Epoch 4/100
3812/3812 [==============================] - 0s 90us/step
Accuracy: 0.0018363064008394543
